In [7]:
from InSituToolkit.imaging_database import write_experiment
import imaging_db.database.db_operations as db_ops
import imaging_db.filestorage.s3_storage as s3_storage
import imaging_db.filestorage.local_storage as local_storage
import imaging_db.utils.db_utils as db_utils
import os, csv


%gui qt5
import numpy as np
from skimage import io
from starfish import Experiment, display, Codebook, ExpressionMatrix, BinaryMaskCollection, LabelImage, FieldOfView
from starfish.image import Filter
from starfish.spots import FindSpots, DecodeSpots, AssignTargets
from starfish.types import Axes, Coordinates, Features, FunctionSource, TraceBuildingStrategies



db_credentials = '/Users/andrew.cote/Documents/db_credentials.json'
list_of_datasets = getIDs(db_credentials, 'SDG')

example_dataset = 'SDG-2019-10-07-22-22-22-0098'

In [8]:
output_dir = "/Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/experiments/" + example_dataset + "/"
exp = Experiment.from_json(output_dir + '/experiment.json')

In [1]:
import pickle

In [10]:
#better to use the FieldOfView.ENUM designator as the string matching to 'primary may change later'

primary_000 = exp['fov_000'].get_images(FieldOfView.PRIMARY_IMAGES)
image_stack = next(primary_000)
display(image_stack)

100%|██████████| 21/21 [00:06<00:00,  3.41it/s]


In [ ]:
# we are basically doing edge detect on gaussian spatial features

ghp = Filter.GaussianHighPass(sigma=3)
high_passed = ghp.run(image_stack, verbose=True, in_place=False)

glp = Filter.GaussianLowPass(sigma=1)
low_passed = ghp.run(high_passed, verbose=True, in_place=False)

In [ ]:
# project z planes by maximum

mproj = Filter.Reduce((Axes.ZPLANE,), func='max', module=FunctionSource.np)
mip = mproj.run(low_passed)

In [ ]:
# find the spots 

p = FindSpots.BlobDetector(
    min_sigma = 1, max_sigma = 10, num_sigma = 10, threshold = 0.001, 
    measurement_type = 'mean'
)